In [1]:
import keplergl
import import_ipynb
import pandas as pd
import helpers

from create_geojson_methods import create_and_get_gdf
from create_geojson_methods import get_initial_map_data
from create_geojson_methods import clean_gdf_columns
from create_geojson_methods import add_initial_map_column_to_gdf
from create_geojson_methods import get_list_of_flip_dict
from create_geojson_methods import implement_flip_into_new_gdf

importing Jupyter notebook from create_geojson_methods.ipynb


In [21]:
if __name__ == '__main__':
    tracts_fname = 'data/tl_2013_55_tract.zip'
    partition_fname = 'data/wi_gerrymander_dem.csv'
    json_fname = 'data/wi_path_1flips.json'
    population_fname = 'data/wi_tract_populations_census_2010.csv'
    voteshares_fname = 'data/wi_voteshares.csv'

    metric_fn_names = ['compute_SL_index', 'compute_efficiency_gap', 'compute_mm_gap']
    
    district_counter = 2

    map_as_gdf = create_and_get_gdf(tracts_fname, partition_fname)
    map_as_gdf = clean_gdf_columns(map_as_gdf)
    map_data_dict = get_initial_map_data(json_fname)
    map_as_gdf = add_initial_map_column_to_gdf(map_data_dict, map_as_gdf)
    
    map_as_gdf = helpers.add_population_data(map_as_gdf, population_fname)
    map_as_gdf = helpers.add_voteshare_data(map_as_gdf, voteshares_fname)
            
    flip_json_list_of_dict = get_list_of_flip_dict(json_fname)
    
    for one_flip_dict in flip_json_list_of_dict:
        new_map_as_gdf = implement_flip_into_new_gdf(one_flip_dict, map_as_gdf, district_counter)
        assignment_dict = {node: new_map_as_gdf['district'][node] for node in new_map_as_gdf.index}
        
        key_list = list(assignment_dict.keys())
        for node in key_list:
            if assignment_dict[node] < 0:
                assignment_dict.pop(node, None)

        new_map_as_gdf.pop('district')
            
        test_partition = helpers.build_partition(new_map_as_gdf, None, assignment_dict)
                
        new_aggregate_map = new_map_as_gdf.dissolve(by='district')

/opt/anaconda3/envs/geo_env/lib/python3.8/site-packages/gerrychain/graph/graph.py:152: UserWarning: Geometry is in a geographic CRS. Results from 'area' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  areas = df.geometry.area.to_dict()


KeyError: 'district'

Compute metrics on partition, save somewhere, then dissolve gdf and add metrics to dissolved gdf; plot

In [19]:
helpers.compute_mm_gap(test_partition)

-0.04502449418085708

In [20]:
helpers.compute_SL_index(test_partition)

	GOP vote shares: [0.5478 0.5335 0.3446 0.342  0.5391 0.5067 0.5366 0.5255]


0.2839332963131571

In [9]:
new_map_as_gdf

,NAMELSAD,lat,lon,geometry,district,plan,population,gop_voteshare,dem_voteshare,gop_votes,dem_votes
GEOID,,,,,,,,,,,
55009940002,Census Tract 9400.02,44.514550,-88.127159,"POLYGON ((-88.15018 44.51202, -88.15010 44.513...",7,2,4729,0.532575,0.467425,2518.546953,2210.453047
55009001600,Census Tract 16,44.488596,-87.983501,"POLYGON ((-88.00246 44.48693, -87.99766 44.488...",7,2,6505,0.532575,0.467425,3464.400069,3040.599931
55009001801,Census Tract 18.01,44.514660,-87.916541,"POLYGON ((-87.95519 44.51945, -87.95512 44.519...",7,2,4971,0.532575,0.467425,2647.430091,2323.569909
55009001802,Census Tract 18.02,44.524604,-87.980108,"POLYGON ((-88.00750 44.53416, -88.00528 44.538...",7,2,5449,0.532575,0.467425,2902.000919,2546.999081
55009002001,Census Tract 20.01,44.497903,-87.959334,"POLYGON ((-87.98522 44.50493, -87.98260 44.505...",7,2,8222,0.532575,0.467425,4378.831264,3843.168736
...,...,...,...,...,...,...,...,...,...,...,...
55043960800,Census Tract 9608,42.711744,-90.623273,"POLYGON ((-90.78594 42.75890, -90.78579 42.759...",3,2,3037,0.488977,0.511023,1485.024209,1551.975791
55043960100,Census Tract 9601,43.085126,-90.533861,"POLYGON ((-90.65769 43.05224, -90.65765 43.052...",3,2,3656,0.488977,0.511023,1787.701188,1868.298812
55043960500,Census Tract 9605,42.806624,-90.867735,"POLYGON ((-91.07932 42.82298, -91.07897 42.825...",3,2,3864,0.488977,0.511023,1889.408477,1974.591523


In [3]:
one_flip_dict

{'55021970700': '3'}

In [6]:
new_map_as_gdf

,NAMELSAD,lat,lon,geometry,district,plan,population,gop_voteshare,dem_voteshare,gop_votes,dem_votes
GEOID,,,,,,,,,,,
55009940002,Census Tract 9400.02,44.514550,-88.127159,"POLYGON ((-88.15018 44.51202, -88.15010 44.513...",7,2,4729,0.532575,0.467425,2518.546953,2210.453047
55009001600,Census Tract 16,44.488596,-87.983501,"POLYGON ((-88.00246 44.48693, -87.99766 44.488...",7,2,6505,0.532575,0.467425,3464.400069,3040.599931
55009001801,Census Tract 18.01,44.514660,-87.916541,"POLYGON ((-87.95519 44.51945, -87.95512 44.519...",7,2,4971,0.532575,0.467425,2647.430091,2323.569909
55009001802,Census Tract 18.02,44.524604,-87.980108,"POLYGON ((-88.00750 44.53416, -88.00528 44.538...",7,2,5449,0.532575,0.467425,2902.000919,2546.999081
55009002001,Census Tract 20.01,44.497903,-87.959334,"POLYGON ((-87.98522 44.50493, -87.98260 44.505...",7,2,8222,0.532575,0.467425,4378.831264,3843.168736
...,...,...,...,...,...,...,...,...,...,...,...
55043960800,Census Tract 9608,42.711744,-90.623273,"POLYGON ((-90.78594 42.75890, -90.78579 42.759...",3,2,3037,0.488977,0.511023,1485.024209,1551.975791
55043960100,Census Tract 9601,43.085126,-90.533861,"POLYGON ((-90.65769 43.05224, -90.65765 43.052...",3,2,3656,0.488977,0.511023,1787.701188,1868.298812
55043960500,Census Tract 9605,42.806624,-90.867735,"POLYGON ((-91.07932 42.82298, -91.07897 42.825...",3,2,3864,0.488977,0.511023,1889.408477,1974.591523
